In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname="skolenie",
    user="admin",
    password="adminadmin",
    host="localhost",
    port="5432"
)

produkty = pd.read_sql_query("SELECT * FROM produkty", conn)
dodavatelia = pd.read_sql_query("SELECT * FROM dodavatelia", conn)
odberatelia = pd.read_sql_query("SELECT * FROM odberatelia", conn)
objednavky = pd.read_sql_query("SELECT * FROM objednavky", conn)
objednavky_produkty = pd.read_sql_query("SELECT * FROM objednavky_produkty", conn)



In [ ]:
import numpy as np

# Celkový počet produktov na sklade
celkovy_pocet = produkty["pocet"].sum()

# Priemerná cena produktu
priemerna_cena = produkty["cena"].mean()

# Najdrahší a najlacnejší produkt
najdrahsi = produkty.loc[produkty["cena"].idxmax()]
najlacnejsi = produkty.loc[produkty["cena"].idxmin()]

print("Celkový počet položiek na sklade:", celkovy_pocet)
print("Priemerná cena produktu: {:.2f} €".format(priemerna_cena))
print("Najdrahší produkt:", najdrahsi["nazov"], "-", najdrahsi["cena"])
print("Najlacnejší produkt:", najlacnejsi["nazov"], "-", najlacnejsi["cena"])


In [ ]:
import matplotlib.pyplot as plt

# Počet produktov v jednotlivých kategóriách
kat_pocet = produkty.groupby("kategoria")["id"].count()
kat_pocet.plot(kind="bar")
plt.xlabel("Kategória")
plt.ylabel("Počet produktov")
plt.title("Počet produktov v jednotlivých kategóriách")
plt.show()

In [ ]:
# Sumarizácia predaného množstva podľa produktu
top_produkty = objednavky_produkty.groupby("produkt_id")["mnozstvo"].sum().sort_values(ascending=False).head(10)
top_produkty = top_produkty.reset_index().merge(produkty[["id", "nazov"]], left_on="produkt_id", right_on="id")
top_produkty.set_index("nazov")["mnozstvo"].plot(kind="bar")
plt.xlabel("Produkt")
plt.ylabel("Predané množstvo")
plt.title("Top 10 najpredávanejších produktov")
plt.show()

In [ ]:
# Prepojenie objednávok, objednavky_produkty a dodavatelia
obj = objednavky.merge(objednavky_produkty, left_on="id", right_on="objednavka_id")
obj = obj.merge(dodavatelia, left_on="dodavatel_id", right_on="id", suffixes=("", "_dodav"))
obj = obj[obj["typ"] == "dodavatelska"]

# Súčet hodnoty objednávok podľa dodávateľa
obj["hodnota"] = obj["cena"] * obj["mnozstvo"]
dodavatelia_trzby = obj.groupby("nazov")["hodnota"].sum().sort_values(ascending=False).head(5)
dodavatelia_trzby.plot(kind="bar")
plt.xlabel("Dodávateľ")
plt.ylabel("Hodnota objednávok (€)")
plt.title("TOP 5 dodávateľov podľa hodnoty objednávok")
plt.show()

In [ ]:
objednavky["datum"] = pd.to_datetime(objednavky["datum"])
objednavky["mesiac"] = objednavky["datum"].dt.to_period("M")
mesacne = objednavky.groupby("mesiac")["id"].count()
mesacne.plot(marker="o")
plt.xlabel("Mesiac")
plt.ylabel("Počet objednávok")
plt.title("Mesačný vývoj počtu objednávok")
plt.show()

In [ ]:
produkty.boxplot(column="cena", by="kategoria", grid=False)
plt.xlabel("Kategória")
plt.ylabel("Cena (€)")
plt.title("Boxplot cien produktov podľa kategórie")
plt.suptitle("")
plt.show()


In [ ]:
# Vyber produkt "mlieko"
produkt_id = produkty[produkty["nazov"] == "mlieko"]["id"].iloc[0]

# Spoj objednávky a produkty, vyber len odberateľské objednávky z posledných 30 dní
import datetime
objednavky["datum"] = pd.to_datetime(objednavky["datum"])
now = objednavky["datum"].max()
start_date = now - pd.Timedelta(days=30)

odberatelske = objednavky[(objednavky["typ"] == "odberatelska") & (objednavky["datum"] >= start_date)]
odberatelske_produkty = odberatelske.merge(objednavky_produkty, left_on="id", right_on="objednavka_id")
mlieko_predaj = odberatelske_produkty[odberatelske_produkty["produkt_id"] == produkt_id]

priemerny_predaj = mlieko_predaj["mnozstvo"].sum() / 30
sklad_zasoby = produkty[produkty["id"] == produkt_id]["pocet"].iloc[0]
dni_na_sklade = sklad_zasoby / priemerny_predaj if priemerny_predaj > 0 else None

print(f"Priemerný denný predaj mlieka: {priemerny_predaj:.2f} ks")
print(f"Zásoby mlieka vystačia na cca {dni_na_sklade:.1f} dní")

In [ ]:
# Prepojenie objednávok, objednavky_produkty a odberatelia
obj = objednavky.merge(objednavky_produkty, left_on="id", right_on="objednavka_id")
obj = obj.merge(odberatelia, left_on="odberatel_id", right_on="id", suffixes=("", "_odber"))
obj = obj[obj["typ"] == "odberatelska"]

obj["trzba"] = obj["cena"] * obj["mnozstvo"]
top_odberatelia = obj.groupby("nazov")["trzba"].sum().sort_values(ascending=False).head(10)
top_odberatelia.plot(kind="bar")
plt.xlabel("Odberateľ")
plt.ylabel("Tržba (€)")
plt.title("TOP 10 odberateľov podľa tržieb")
plt.show()

In [ ]:
conn.close()